### Dota Hero Grid Editor

Json file in steam folder:\
`C:\Program Files (x86)\Steam\userdata\YOUR_FRIEND_ID\570\remote\cfg\hero_grid_config.json`

let's copy it into our current folders to play with

In [ ]:
from dota_utils import *

import shutil
from config import DOTA_FRIENDID

steam = f'C:\\Program Files (x86)\\Steam\\userdata\\{DOTA_FRIENDID}\\570\\remote\\cfg'
json_name = '\\hero_grid_config.json'
src = steam + json_name
dst = '.'
dst_saved = './saved' # just for fail-safe reasons
shutil.copy2(src, dst)
shutil.copy2(src, dst_saved)

###

In [2]:
import json
# READ FROM FILE
with open('hero_grid_config.json') as json_file:
    data = json.load(json_file)

In [5]:
# WRITE INTO FILE 
with open('hero_grid_config.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
# copy file back to Steam folder when we are done
src_laptop = dst + json_name
shutil.copy2(src_laptop, steam)
'' # so it  does not reveal my id xd

### Info
```python
max_x = 1200
max_y = 598
```
`data['configs'][0]` is my main grid\
`data['configs'][1]` is Dota Plus levels grid

### Dota Plus Levels Grid

In [39]:
dict_help = { # top_left_x, top_left_y, width, height
    'Grandmaster': [0, 0, 300, 100],
    'Master':      [300, 0, 300, 100],
    'Platinum':    [600, 0, 600, 100],
    'Gold':        [0, 120, 600, 100],
    'Silver':      [600, 120, 600, 100],
    'Bronze':      [0, 240, 1200, 275 + 74],
}

for cat in data['configs'][1]['categories']:
    array = dict_help[cat['category_name']]
    cat['x_position'] = array[0]
    cat['y_position'] = array[1]
    cat['width'] = array[2]
    cat['height'] = array[3]

In [189]:
data['configs'][0]['categories'][-1]

{'category_name': 'bans',
 'x_position': 925,
 'y_position': 529.0,
 'width': 275,
 'height': 69.0,
 'hero_ids': [137, 88]}

### My default picking screen Grid

In [ ]:
names = []
for i in data['configs'][0]['categories'][0]['hero_ids']:
    names.append(name_by_id(i))
    
new_names = sorted(names, key=str.casefold)

new_ids = []
for i in new_names:
    new_ids.append(id_by_name(i))
    
data['configs'][0]['categories'][0]['hero_ids'] = new_ids
print(new_ids)

In [4]:
max_x = 1200
max_y = 598

height = 100
delta_y = (max_y - 5 * height)/4

pos_w = 6 * 58 # 7 for six heroes in pos 1
others_x = pos_w + 1

counter = 0

def give_xywh(item, *, x, y, w, h):
    item['x_position'] = x
    item['y_position'] = y
    item['width'] = w
    item['height'] = h


categories = data['configs'][0]['categories']
for counter, item in enumerate(categories[0:5]):
    give_xywh(
        item,
        x=0,
        y=0 + (height + delta_y) * counter,
        w=pos_w,
        h=height
    )

ban_height = 70
delta_others_y = 10
ban_delta = 0
for counter, item in enumerate(categories[6:9]):
    give_xywh(
        item,
        x=others_x ,
        y=counter * (max_y - ban_height)/3,
        w=max_x - others_x ,
        h=(max_y - ban_height - ban_delta - 2 * delta_others_y)/3
    )


give_xywh(
    categories[5],
    x=max_x - (max_x - others_x)/2,
    y=max_y - ban_height - 2,
    w=(max_x - others_x)/2,
    h=ban_height
)

give_xywh(
    categories[9],
    x=others_x,
    y=max_y - ban_height - 2,
    w=(max_x - others_x)/2,
    h=ban_height
)